In [1]:
import numpy as np
from scipy.stats import norm 
from scipy.stats import bernoulli
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import sys
import csv
import math
import copy
import os

In [2]:
# Global Variables

kDim = 5 # the number of dimension
# kUsers = 610 # 610 is total, but we use 300 users 
# kMovies = 9724 # 9724 is total, but we use 6000 movies

# These data are used to draw a kDim-vector that follows Gaussian Distribution (kMean, kCov)
kMean = []
for i in range(kDim):
    kMean.append(0)
kCov = 0.1 * np.identity(n=kDim,dtype='float')

# Rating matrix, which is sparsew
gR = dict() # Key = (userId, movieId)
gD = dict() # Key = (userId, moveId) value = existence
gU = dict() # Key = (userId), value = user vector
gV = dict() # Key = (movieId), value = movie vector

In [3]:
# Generate all model variables reading given file(csv)
# It should be executed first before calling MakeTestSet which split the whole file into 
# training file and test file
def GenerateModelVariables (in_file_name, in_U, in_V):
    file = open(in_file_name, 'r')
    fileReader = csv.reader(file)
    
    for row in fileReader:
        if row[0] == 'userId':
            continue
        else:
            currentUserID = int(row[0])
            currentMovieID = int(row[1])
            currentRating = float(row[2])

            if type(in_U.get(currentUserID)) == type(None):
                in_U[currentUserID] = np.random.multivariate_normal(kMean, kCov, 1).T
            if type(gV.get(currentMovieID)) == type(None):
                in_V[currentMovieID] = np.random.multivariate_normal(kMean, kCov, 1).T

In [4]:
# Make test set... from given training file
def MakeTestSet(in_file_name, in_ratio, out_trainingFileName, out_testFileName):
    originalFile = open(in_file_name, 'r')
    fileReader = csv.reader(originalFile)
    
    trainingFile = open(out_trainingFileName,'w')
    testFile = open(out_testFileName, 'w')
    
    trainingCSVWriter = csv.writer(trainingFile, delimiter=',',quotechar=',', quoting=csv.QUOTE_MINIMAL)
    testCSVWriter = csv.writer(testFile, delimiter=',',quotechar=',', quoting=csv.QUOTE_MINIMAL)
    
    trainingCSVWriter.writerow(['userId', 'movieId', 'rating'])
    testCSVWriter.writerow(['userId', 'movieId', 'rating'])
    
    for row in fileReader:
        if row[0] == 'userId':
            continue
        else:
            bTrainingSet = bernoulli.rvs(in_ratio, size = 1)[0] # 1 indicates training data
            if bTrainingSet == 1:
                trainingCSVWriter.writerow([row[0], row[1], row[2]])
            else:
                testCSVWriter.writerow([row[0], row[1], row[2]])

In [5]:
# Read training file
# This function extract all information on the set of given ratings and existence
def ReadTrainingCSVFile(in_file_name, out_D, out_R):
    training_file = open(in_file_name, 'r')
    fileReader = csv.reader(training_file)
    
    for row in fileReader:
        if row[0] == 'userId':
            continue
        else:            
            currentUserID = int(row[0])
            currentMovieID = int(row[1])
            currentRating = float(row[2])
                        
            out_D[(currentUserID, currentMovieID)] = 1
            out_R[(currentUserID, currentMovieID)] = currentRating
            

In [6]:
# It is used to verify that the objective function is minimizing
def CalculateJointLikelihood(in_R, in_U, in_V, in_D):
    
    logLikelihood = 0.0
    
    for eachKey in in_D.keys():
        logLikelihood = logLikelihood + ((in_R[eachKey] - (in_U[eachKey[0]].T @ in_V[eachKey[1]])) ** 2)
    
    return logLikelihood

In [7]:
# We solve this problem iteratively. 
# For example, first set all U, V = 0
#
# For the first iteration:
# Calculate U using given V
# Next, 
# Calculate V using calculated U above.

def CalculateMLE(in_init_U, in_init_V, in_D, in_R, in_nIterations):
    
    U = copy.deepcopy(in_init_U) # init U
    V = copy.deepcopy(in_init_V) # init V
    
    for t in range(in_nIterations):
        for i in U.keys():
            firstTerm = np.zeros([kDim, kDim], float)
            secondTerm = np.zeros([kDim, 1], float)

            for j in V.keys():
                if type(in_D.get((i,j))) != type(None):
                    
                    firstTerm = firstTerm + (V[j] @ V[j].T)
                    secondTerm = secondTerm + (in_R[(i, j)] * V[j])
            
            U[i] = np.linalg.pinv(firstTerm) @ secondTerm

        for j in V.keys():
            firstTerm = np.zeros([kDim, kDim], float)
            secondTerm = np.zeros([kDim, 1], float)

            for i in U.keys():
                if type(in_D.get((i,j))) != type(None):
                    
                    firstTerm = firstTerm + (U[i] @ U[i].T)
                    secondTerm = secondTerm + (in_R[(i, j)] * U[i])
            
            V[j] = np.linalg.pinv(firstTerm) @ secondTerm
        
        # calculate current joint likelihood
        currentLikelihood = CalculateJointLikelihood(in_D=gD, in_R= gR, in_U=U, in_V=V)
        
        # Show the current value of objective function
        print("Value => ", currentLikelihood)
        
    return U, V

In [8]:
def GetSample_UV(in_init_U, in_init_V, in_R, in_D, in_nSamples, in_stepSize):
    
    U = copy.deepcopy(in_init_U) # init U
    V = copy.deepcopy(in_init_V) # init V
    
    nUsers = len(U)
    nMovies = len(V)
    c = 1.0
    
    u_samples = []
    v_samples = []
    
    for n in range(in_nSamples):
        for t in range(in_stepSize):
            print("iteration => ", t)
            for i in U.keys():
                firstTerm = np.identity(n=kDim,dtype='float') / c
                secondTerm = np.zeros([kDim,1],dtype='float')

                for j in V.keys():
                    if type(in_D.get((i,j))) != type(None):
                        firstTerm = firstTerm + (V[j] @ V[j].T)
                        secondTerm = secondTerm + (in_R[(i, j)] * V[j])

                covTerm = np.linalg.inv(firstTerm)
                muTerm = (covTerm @ secondTerm).T

                U[i] = np.random.multivariate_normal(muTerm[0], covTerm, 1).T

            for j in V.keys():
                firstTerm = np.identity(n=kDim,dtype='float') / c
                secondTerm = np.zeros([kDim,1],dtype='float')

                for i in U.keys():
                    if type(in_D.get((i,j))) != type(None):

                        firstTerm = firstTerm + (U[i] @ U[i].T)
                        secondTerm = secondTerm + (in_R[(i, j)] * U[i])

                covTerm = np.linalg.inv(firstTerm)
                muTerm = (covTerm @ secondTerm).T

                V[j] = np.random.multivariate_normal(muTerm[0], covTerm, 1).T
        
        print("We've got one sample from full posterior distribution")
        u_samples.append(U)
        v_samples.append(V)
    return u_samples,v_samples

In [9]:
def GetAllErrors_GibbsVersion(in_testFileName, in_D, in_R, in_init_U, in_init_V, in_nSamples):
    
    USamples, VSamples = GetSample_UV(in_D=in_D,
                                      in_R=in_R,
                                      in_init_U=in_init_U,
                                      in_init_V=in_init_V,
                                      in_nSamples=in_nSamples,
                                      in_stepSize=10)
    
    trainingSquaredErrorSum = 0.0
    testSquaredErrorSum = 0.0
    
    ##### training error
    for eachKey in in_D.keys():
        sum = 0.0
        for n in range(in_nSamples):
            sum = sum + (USamples[n][eachKey[0]].T @ VSamples[n][eachKey[1]])[0][0]
        sum = sum / float(in_nSamples)
        trainingSquaredErrorSum = trainingSquaredErrorSum + ((in_R[eachKey] - sum) ** 2)
    
    trainingSquaredErrorSum = trainingSquaredErrorSum / len(gR)
    print('Training Error (Gibbs)', trainingSquaredErrorSum)
    
    #### test error
    test_file = open(in_testFileName, 'r')
    fileReader = csv.reader(test_file)
    
    testSquaredErrorSum = 0.0
    testSetCount = 0
    for row in fileReader:
        if row[0] == 'userId':
            continue
        else:
            testSetCount = testSetCount + 1
            currentUserID = int(row[0])
            currentMovieID = int(row[1])
            currentRating = float(row[2])
                
            sum = 0.0
            for n in range(in_nSamples):
                sum = sum + (USamples[n][currentUserID].T @ VSamples[n][currentMovieID])[0][0]
            estimatedRating = sum / float(in_nSamples)
            
            testSquaredErrorSum = testSquaredErrorSum + ((estimatedRating - currentRating) ** 2)
    
    testSquaredErrorSum = testSquaredErrorSum / float(testSetCount)
    print('Test Error (Gibbs)', testSquaredErrorSum)
    
    return trainingSquaredErrorSum, testSquaredErrorSum
    

In [10]:
def GetAllErrors_MLEVersion(in_testFileName, in_D, in_R, in_init_U, in_init_V, in_nIterations):
    
    U, V = CalculateMLE(gU, gV, gD, gR, in_nIterations)
    trainingSquaredErrorSum = 0.0
    testSquaredErrorSum = 0.0
    
    ##### training error
    for eachKey in in_D.keys():
        trainingSquaredErrorSum = trainingSquaredErrorSum + (((U[eachKey[0]].T @ V[eachKey[1]])[0][0] - in_R[eachKey]) ** 2)
    
    trainingSquaredErrorSum = trainingSquaredErrorSum / len(gR)
    print('Training Error (MLE)', trainingSquaredErrorSum)
    
    #### test error
    test_file = open(in_testFileName, 'r')
    fileReader = csv.reader(test_file)
    
    testSquaredErrorSum = 0.0
    testSetCount = 0
    testErrorList = []
    for row in fileReader:
        if row[0] == 'userId':
            continue
        else:
            testSetCount = testSetCount + 1
            currentUserID = int(row[0])
            currentMovieID = int(row[1])
            currentRating = float(row[2])
                
            estimatedRating = (U[currentUserID].T @ V[currentMovieID])[0][0]
            testSquaredErrorSum = testSquaredErrorSum + ((estimatedRating - currentRating) ** 2)
            testErrorList.append(((estimatedRating - currentRating) ** 2))
    
    testSquaredErrorSum = testSquaredErrorSum / float(testSetCount)
    print('Test Error (MLE)', testSquaredErrorSum)
    
    return trainingSquaredErrorSum, testSquaredErrorSum, testErrorList
    

In [11]:
# clear
gU.clear()
gV.clear()
gD.clear()
gR.clear()

# Get all user id and movie id in order to avoid encontering unseen user or movie 
GenerateModelVariables (in_file_name='movie_ratings.csv', in_U=gU, in_V=gV)

In [ ]:
splitList = [0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.99]
splitIndex = ['65%', '70%', '75%', '80%', '85%', '90%', '95%', '99%']

MLEMSETestErrorsLists = [] # List of List
MLETrainingMSE_List = []
MLETestMSE_List = []

gibbsTrainingMSE_List = []
gibbsTestMSE_List = []


for i in range(len(splitList)):
    # Split the movie_ratings.csv file into traingSet.csv, testSet.csv
    MakeTestSet('movie_ratings.csv', splitList[i], 'trainingSet.csv', 'testSet.csv')
    
    # Read training data file to read given ratings.
    ReadTrainingCSVFile(in_file_name="trainingSet.csv", out_D=gD,out_R=gR)

    # For gibbs model
    currentGibbsTrainingMSE = 0.0
    currentGibbsTestMSE = 0.0
    
    currentGibbsTrainingMSE, currentGibbsTestMSE = GetAllErrors_GibbsVersion(in_D=gD, in_R=gR,in_init_U=gU,in_init_V=gV,in_nSamples=10,in_testFileName='testSet.csv')
    gibbsTrainingMSE_List.append(currentGibbsTrainingMSE)
    gibbsTestMSE_List.append(currentGibbsTestMSE)
    
    # For MLE model
    currentMLETrainingMSE = 0.0
    currentMLETestMSE = 0.0
    currentMLEMSETestErrorLists = []
    
    currentMLETrainingMSE, currentMLETestMSE, currentMLEMSETestErrorLists = GetAllErrors_MLEVersion(in_D=gD, in_R=gR, in_init_U=gU, in_init_V=gV,in_testFileName='testSet.csv',in_nIterations=30)
    MLETrainingMSE_List.append(currentMLETrainingMSE)
    MLETestMSE_List.append(currentMLETestMSE)
    MLEMSETestErrorsLists.append(currentMLEMSETestErrorLists)
    
    # Remove 'trainingSet.csv', and 'testSet.csv' files for the next experiment
    os.remove("trainingSet.csv")
    os.remove("testSet.csv")
    print('One Split Done')
    

iteration =>  0
iteration =>  1
iteration =>  2
iteration =>  3
iteration =>  4
iteration =>  5
iteration =>  6
iteration =>  7
iteration =>  8
iteration =>  9
We've got one sample from full posterior distribution
iteration =>  0
iteration =>  1
iteration =>  2
iteration =>  3
iteration =>  4
iteration =>  5
iteration =>  6
iteration =>  7
iteration =>  8
iteration =>  9
We've got one sample from full posterior distribution
iteration =>  0
iteration =>  1
iteration =>  2
iteration =>  3
iteration =>  4
iteration =>  5
iteration =>  6
iteration =>  7
iteration =>  8
iteration =>  9
We've got one sample from full posterior distribution
iteration =>  0
iteration =>  1
iteration =>  2
iteration =>  3
iteration =>  4
iteration =>  5
iteration =>  6
iteration =>  7
iteration =>  8
iteration =>  9
We've got one sample from full posterior distribution
iteration =>  0
iteration =>  1
iteration =>  2
iteration =>  3
iteration =>  4
iteration =>  5
iteration =>  6
iteration =>  7
iteration =>  8


In [ ]:
MLEMSETestErrorsLists # List of List
print("MLE Training", MLETrainingMSE_List)
print("MLE Test", MLETestMSE_List)

gibbsTrainingMSE_List
print("Gibbs Training",gibbsTrainingMSE_List)
print("Gibbs Test",gibbsTestMSE_List)


In [ ]:
plt.figure(figsize=(10,7))

plt.plot(MLETestMSE_List ,'o--', color='darkgreen' )
plt.plot(MLETrainingMSE_List, 'o--', color='blue')
plt.xticks(range(len(splitIndex)), splitIndex)


In [ ]:
plt.figure(figsize=(10,7))

gibbsTrainingMSE_List = []
gibbsTestMSE_List = []

plt.plot(gibbsTrainingMSE_List ,'o--', color='darkgreen')
plt.plot(gibbsTestMSE_List, 'o--', color='blue')
plt.xticks(range(len(splitIndex)), splitIndex)
